In [1]:
!pip install ultralytics==8.0.196
!pip install roboflow

In [2]:
from ultralytics import YOLO
import os
import cv2
import glob
import shutil
import matplotlib.pyplot as plt
import numpy as np
from roboflow import Roboflow
import json

In [ ]:
with open('config.json') as config_file:
    config = json.load(config_file)

# Access the API key
api_key = config.get('ROBOFLOW_API_KEY')

In [3]:

rf = Roboflow(api_key=api_key)
project = rf.workspace("label-my-cats-images").project("cat_feeder_project")
version = project.version(9)
dataset = version.download("yolov8")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to cat_feeder_project-9 in yolov8:: 100%|██████████| 14898/14898 [00:02<00:00, 7323.72it/s]


In [4]:
dataset_dir = dataset.location

train_img_dir = os.path.join(dataset_dir, 'train', 'images')
train_label_dir = os.path.join(dataset_dir, 'train', 'labels')
valid_img_dir = os.path.join(dataset_dir, 'valid', 'images')
valid_label_dir = os.path.join(dataset_dir, 'valid', 'labels')
test_img_dir = os.path.join(dataset_dir, 'test', 'images')
test_label_dir = os.path.join(dataset_dir, 'test', 'labels')


In [5]:
data_yaml_path = '/content/cat_feeder_project-9/data.yaml'

print(f"data_yaml_img_dir: {data_yaml_path}")

data_yaml_img_dir: /content/cat_feeder_project-9/data.yaml


In [6]:
model = YOLO("yolov8s.yaml")

model.train(
    data=data_yaml_path,
    epochs=50,
    imgsz=640,
    batch=32,
    workers=4,
    )


                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e70fbc55d20>
fitness: 0.8046819307561197
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.86667,     0.86486,     0.62953])
names: {0: 'Niuniu', 1: 'Orange', 2: 'human'}
plot: True
results_dict: {'metrics/precision(B)': 0.9455829806670834, 'metrics/recall(B)': 0.959536582236185, 'metrics/mAP50(B)': 0.9636425481329175, 'metrics/mAP50-95(B)': 0.7870196399364754, 'fitness': 0.8046819307561197}
save_dir: PosixPath('runs/detect/train')
speed: {'preprocess': 0.24945120657643963, 'inference': 4.755917672188051, 'loss': 0.0005275972427860383, 'postprocess': 5.815472910481114}

In [7]:
metrics = model.val(data=data_yaml_path)

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8s summary (fused): 168 layers, 11126745 parameters, 0 gradients, 28.4 GFLOPs
val: Scanning /content/cat_feeder_project-9/valid/labels.cache... 620 images, 41 backgrounds, 0 corrupt: 100%|██████████| 620/620 [00:00<?, ?it/s]
os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 20/20 [00:13<00:00,  1.43it/s]
                   all        620        621      0.946       0.96      0.964      0.787
                Niuniu        620        289      0.997       0.99      0.995      0.867
                Orange        620        312      0.987      0.989      0.995      0.866
                 human        620         20      0.852        0.9      0.901      0.629
Speed: 0.3ms preprocess, 10.0ms inference, 0.0ms 

In [8]:
version.deploy("yolov8", "/content/runs/detect/train")

View the status of your deployment at: https://app.roboflow.com/label-my-cats-images/cat_feeder_project/9
Share your model with the world at: https://universe.roboflow.com/label-my-cats-images/cat_feeder_project/model/9
